**Table of contents**<a id='toc0_'></a>    
- [Regularization in the linear model](#toc1_)    
  - [The dataset](#toc1_1_)    
    - [Data loading and characterization](#toc1_1_1_)    
    - [Separating features and response](#toc1_1_2_)    
    - [Creating a training/test partition](#toc1_1_3_)    
  - [Regularization in the linear regression](#toc1_2_)    
    - [Regression in high dimensions](#toc1_2_1_)    
    - [Regularization: Lasso, ridge and elastic net](#toc1_2_2_)    
  - [Benchmark: linear regression (OLS)](#toc1_3_)    
  - [Ridge regression](#toc1_4_)    
    - [Visualizing coefficients paths](#toc1_4_1_)    
    - [Selection of the tuning parameter through cross-validation](#toc1_4_2_)    
    - [Estimating the test error](#toc1_4_3_)    
  - [LASSO regression](#toc1_5_)    
    - [Estimated paths](#toc1_5_1_)    
    - [Tuning the model through CV](#toc1_5_2_)    
    - [Test error estimation](#toc1_5_3_)    
  - [YOUR TURN](#toc1_6_)    
  - [YOUR TURN 2](#toc1_7_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Regularization in the linear model](#toc0_)


We start by importing the necessary libraries for this lab:

In [3]:
# Next three lines load complete libraries
import random
import numpy as np
import pandas as pd
import warnings

# This one imports just one module (pyplot) from library matplotlib
import matplotlib.pyplot as plt

# Then, we import specific functions from different sklearn modules 
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, RidgeCV, ElasticNet, ElasticNetCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

# Finally we set a filter to avoid verbosity in warnings
warnings.filterwarnings("ignore")
# and a random seed to ensure reproducibility 
# Note setting the random seed at this point requires the notebook to be run sequentially (keeping always the same order of code lines)
random.seed(1) # This sets 

## 

## <a id='toc1_1_'></a>[The dataset](#toc0_)

### <a id='toc1_1_1_'></a>[Data loading and characterization](#toc0_)

We load a synthetic dataset with 400 predictors and a reponse variable. The coefficient structure of the underlying model is sparse (most coefficients are approximately zero) with only a few having an impact on the response. We load the dataset using the `read_csv` function in pandas: 

In [4]:
url = 'juandmontoro.github.io/bigDataEco/data/regularized_regression.csv'
data = pd.read_csv(url)

In [ ]:
data

We also load the coefficients that generated the data:

In [ ]:
betas = pd.read_csv('betas.csv')
betas['0']

Features are sorted so the first one has the most impact on the response and it decays 

We can plot the decaying pattern of the actual coefficients of the DGM and see that less than 30 of them are greater than $10^{-2}$:

In [ ]:
plt.plot(betas['0'])
plt.yscale('log')
plt.axhline(y=10**(-2),color='r',linestyle='--')
plt.show()

In [ ]:
sum(betas['0']>10**-2)

### <a id='toc1_1_2_'></a>[Separating features and response](#toc0_)

Let us split the dataframe into design matrix and response:

In [7]:
X = data.filter(like='Feature')
y = data.filter(like='y')

We can take a look at the resulting design matrix:

In [ ]:
X

And the response:

In [ ]:
y

### <a id='toc1_1_3_'></a>[Creating a training/test partition](#toc0_)

We partition the dataset into training and test (20%) samples. To do so we use the Python-feature called "tuple unpacking". When a function returns  a tuple (which is an iterable) you can assign multiple variables at once on the left-hand side of an assignment. See the code next:

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train

In [ ]:
y_train

The output shows that the data has been shuffled (but indices in both features and response match) and only 120 observations have been selected. You can check that `X_test` and `y_test` also match.

## <a id='toc1_2_'></a>[Regularization in the linear regression](#toc0_)

### <a id='toc1_2_1_'></a>[Regression in high dimensions](#toc0_)

 **Note** the regression problem we are considering is to estimate the conditional mean of the response,  $$E[Y|X]=\hat{f}(X)$$ is is a high-dimensional  one, as $k>>n$. This would fail in the standard linear regression setting as there are many (infinite) regression lines that could be fit to the dataset (think of fitting a line to a point).

Penalized regression can estimate a model even if $k>n$ through **coordinate descent**: the objective function can be minimized iteratively, one parameter (or coordinate) at a time, while keeping all others fixed.

How It Works:

1. Objective Function: Consider a loss function L(β1,β2,…,βp) to be minimized with respect to β=(β1,β2,…,βp).

2. Iterative Updates:
    - Fix all coefficients except for one, say βj.
    - Minimize f with respect to βj​ while keeping all other coefficients constant.
    - Repeat this process for all coordinates (β1,β2,…,βpβ1​,β2​,…,βp​) cyclically or in some order.

3. Repeat Until Convergence: The algorithm cycles through all coordinates multiple times until the changes in the coefficients (or the value of the objective function) become negligible.

### <a id='toc1_2_2_'></a>[Regularization: Lasso, ridge and elastic net](#toc0_)


Elastic Net is a regularization technique that combines both Lasso (Least Absolute Shrinkage and Selection Operator) and Ridge regression. It achieves this by introducing two penalty terms to the loss function: one for Lasso and one for Ridge. The regularization term is

$$
\lambda \left( \alpha \sum_{j=1}^{p} | \beta_j | + \frac{1 - \alpha}{2} \sum_{j=1}^{p} \beta_j^2 \right)
$$

Here:
- $\lambda$ is the regularization parameter that controls the overall strength of the penalty.
- $\alpha$ is the mixing parameter that balances between Lasso ($\alpha = 1$) and Ridge ($\alpha = 0$).
- $\beta_j$ are the coefficients of the model.

Specifically, the Elastic Net penalty is a linear combination of the L1 norm (used in Lasso) and the L2 norm (used in Ridge). When the mixing parameter $\alpha$ is set to 1, Elastic Net behaves like Lasso, applying only the L1 penalty, which encourages sparsity by shrinking some coefficients exactly to zero. Conversely, when $\alpha$ is set to 0, it behaves like Ridge regression, applying only the L2 penalty, which shrinks coefficients uniformly but does not enforce sparsity. By tuning $\alpha$, Elastic Net can balance between these two extremes, leveraging the strengths of both methods.

Note: The division by 2 in the term $\frac{1 - \alpha}{2}$ is actually a standard convention in the formulation of the Elastic Net penalty. This is done to ensure that the regularization term is properly scaled and comparable to the L1 penalty term.

## <a id='toc1_3_'></a>[Benchmark: linear regression (OLS)](#toc0_)

In [ ]:
lm = LinearRegression()
lm.fit(X_train,y_train)

We can see the in-sample predictive performance (perfect):

In [ ]:
lm.score(X_train,y_train)

In [ ]:
mean_squared_error(y_true=y_train,y_pred=lm.predict(X_train))

However the test error is not that good:

In [ ]:
lm.score(X_test,y_test)

In [ ]:
mean_squared_error(y_true=y_test,y_pred=lm.predict(X_test))

## <a id='toc1_4_'></a>[Ridge regression](#toc0_)

Ridge regression introduces a L2 penalty to the standard least squares loss function.
The ridge regression objective function is:
$$
\text{Minimize} \left( \sum_{i=1}^n (y_i - \sum_{j=1}^p x_{ij}\beta_j)^2 + \lambda \sum_{j=1}^p \beta_j^2 \right)
$$

The term $\lambda$ is referred to as the penalization parameter. This parameter controls the amount of regularization applied to the model. Specifically, it adds a penalty to the size of the coefficients in the regression model, which helps to prevent overfitting by shrinking the coefficients towards zero. In other words, a larger $\lambda$ increases the amount of shrinkage, leading to smaller coefficient estimates.

As the coefficients change depending on the amount of the penalty we start with a visualization of the coefficients path against the penalization parameter $\lambda$

### <a id='toc1_4_1_'></a>[Visualizing coefficients paths](#toc0_)

We start by creating a grid of lambda values on which estimate the coefficients. We start creating 100 lambda values (in a log scale):

In [ ]:
lambdas = np.logspace(-5,1,100) # we create 100 values between -1 and 2; then convert each to log10(value) 
lambdas

Next, we estimate the ridge regression coefficients for the different values of lambda. To do so we need to use the function `ElasticNet()` from the module `sklearn.linear_model`.  Note that a ridge regression is an elastic net regression with L1 (LASSO) regularization set to 0. 

In addition, note that:

- In sklearn regularized regression models the parameter $\lambda$  is called `alpha` (or `alphas` if more than one value is passed) in `sklearn`. In order to be consistent with our previous discussion we refer to lambdas rather than alphas.
- In Elastic net the argument `l1_ratio` can be between 01 and 1; 0 for ridge regression, 1 for LASSO. Any value in between introduces L1+L2 penalty. In this example we will set `l1_ratio` to 0.
- To fit a path of ridge regression models we use `path()` which can fit both ridge and lasso, as well as a hybrid mixture, across different $\lambda$ values
- It is good practice to standardize the columns of X in these applications, if the variables are measured in different units, as the ridge or lasso penalty is affected by the magnitude of the coefficients. We will introduce this a later stage. 




In [14]:

alphas, coefs, _ = ElasticNet.path(X_train,y_train,l1_ratio =0.,alphas=lambdas) 

# The argument l1_ratio controls for the type of penalty used
# l1_ratio : float, default=0.5 Number between 0 and 1 passed to elastic net (mix of l1 and l2 penalties). 
# l1_ratio=1 corresponds to the Lasso; 
# l1_ratio=0 corresponds to ridge regression; 


# Again we are doing unpacking in this code.
#  The path function returns alphas, coeficients and dual gaps (a measure of how close the 
# current solution is to the optimal solution of the Elastic Net optimization problem).
# We only need the former two, hence the use of an underscore _ (means we will not save that value)

We can print both the lambdas (which is unnecessary as they were provided by us) and the associated coefficients:

In [ ]:
alphas

In [ ]:
coefs

Important: what is the dimensionality of the coefficients? We have three indexes (the first refers to the response variable, only one in this case; the second to the coefficient, 400; the third to the value of lambda, 100):

In [ ]:
coefs.shape

We are going to remove the first dimension (as only one response y is considered in this setup):

In [ ]:
coefs = coefs[0,:,:]
coefs.shape

Next we create a dataframe with the solutions path including coefficient values indexed by the lambdas:

In [19]:
soln_path = pd.DataFrame (coefs.T,
                           columns=X_train.columns,
                           index=np.log10(alphas))
soln_path.index.name = 'log(lambda)'

In [ ]:
soln_path

**We see that as lambda increases the coefficients are shrinked towards zero!**

Finally we plot this path:

In [ ]:
plt.figure(figsize=(10,10))
ax = soln_path.plot(legend=False)
ax.set_xlabel ('$\log_{10}(\lambda)$', fontsize =20)
ax.set_ylabel ('Coefficients ', fontsize =20)
plt.show()

The plot shows how the coefficients shrink towards zero as we increase the penalty $\lambda$ (only for lambda=infinity they become zeroes).

### <a id='toc1_4_2_'></a>[Selection of the tuning parameter through cross-validation](#toc0_)

Which value is better? It is difficult to know. To choose the best value we will proceed with cross validation (CV). The ridge, lasso, and elastic net can be efficiently fit along a sequence of λ values, creating what is known as a solution path or regularization path. Hence there is specialized code to fit such paths, and to choose a suitable value of λ using cross-validation. 

In [ ]:
ridgeCV = RidgeCV(alphas=lambdas,      
                  cv=5, # we use 5-fold CV; if not set ridge performs LOOCV (solution will likely change)
                  scoring='neg_mean_squared_error' # we use -MSE to choose tuning parameter
                )
ridgeCV.fit(X_train,y_train)

The cross-validated $\lambda$ is

In [ ]:
ridgeCV.alpha_

> Note: the value returned by the above expression is a numpy's double-precision floating-point object. Although unnecessary (this representation is compatible with Python), one can retrieve the number using the method `.item()` 

In [ ]:
ridgeCV.alpha_.item()

And the set of coefficients:

In [ ]:
ridgeCV.coef_

We can also retrieve the fitted intercept in the final model:

In [ ]:
ridgeCV.intercept_

### <a id='toc1_4_3_'></a>[Estimating the test error](#toc0_)

Finally we get the score ($R^2$) and MSE for the trained dataset:

In [ ]:
ridgeCV.score(X_train,y_train)

In [ ]:
mean_squared_error(y_train,ridgeCV.predict(X_train))

But most important, we get the score and mse for the test dataset:

In [ ]:
ridgeCV.score(X_test,y_test)

In [ ]:
mean_squared_error(y_test,ridgeCV.predict(X_test))

Or we could try the process by hand as follows:

In [ ]:
r2_score(y_test,ridgeCV.predict(X_test))

As expected, the test error (or the $R^2$ score for that matter) is significantly larger ($R^2$ smaller) than than the training (in-sample) counterparts.

Let us store the different results to generate at the end of the activity a summary table:

In [ ]:
# we create empty lists for the different metrics
model = []
r2_train = []
mse_train = []
r2_test =[]
mse_test = []

# Next we append the specific values obtained
model.append('Ridge')
r2_train.append(ridgeCV.score(X_train,y_train))
r2_test.append(ridgeCV.score(X_test,y_test))
mse_train.append(mean_squared_error(y_train,ridgeCV.predict(X_train)))
mse_test.append(mean_squared_error(y_test,ridgeCV.predict(X_test)))


## <a id='toc1_5_'></a>[LASSO regression](#toc0_)

### <a id='toc1_5_1_'></a>[Estimated paths](#toc0_)

Let us first, see the LASSO coefficient paths for different $\lambda$ values:

In [33]:
lambdas = np.logspace(-5,1,100)

In [34]:
alphas, coefs, _ = ElasticNet.path(X_train,y_train,l1_ratio =1,alphas=lambdas)
coefs=coefs[0,:,:] 

In [35]:
soln_path = pd.DataFrame (coefs.T,
                           columns=X_train.columns,
                           index=np.log10(alphas))
soln_path.index.name = 'log Lambda'

In [ ]:
soln_path

We see selection in action (more features are forced down to zero as $\lambda$ increases). Let us take a look at the plot of the paths:

In [ ]:
plt.figure(figsize=(10,10))
ax = soln_path.plot(legend=False)
ax.set_xlabel ('$\log(\lambda)$', fontsize =20)
ax.set_ylabel ('Coefficients ', fontsize =20)
plt.show()

### <a id='toc1_5_2_'></a>[Tuning the model through CV](#toc0_)

Again to decide on a $\lambda$ value we perform hyperparameter tuning through cross-validation: 

In [ ]:
lassoCV = LassoCV(alphas=lambdas, 
                       cv=5)
lassoCV.fit(X_train, y_train)

Note that `LassoCV` could alternatively take the number of lambdas instead of a list of values:

````{python}
    lassoCV = LassoCV(n_alphas=100, 
                       cv=5)

````
This is not an option in `RidgeCV`.

Once we run cross-validation, we can retrieve the best `lambda`:

In [ ]:
lassoCV.alpha_

And the coefficients:

In [ ]:
lassoCV.coef_

We see variable selection feature in LASSO:

In [ ]:
np.count_nonzero(lassoCV.coef_)

`LassoCV` also stores the MSE paths for the cross-validated resamplings for each value of lambda (again this is not the case in `RidgeCV`). Let us create a dataframe with all the five values of the MSE across lambda iterations:

In [ ]:
pd.DataFrame(lassoCV.mse_path_,
             index = ['log(lambda)='+str(round(np.log10(num),3)) for num in lambdas])

We can compute the mean for each the 5 folds across all lambda values:

In [ ]:
pd.DataFrame(lassoCV.mse_path_).mean(1)
# mean(1) indicates to average values across columns. 
# Change to 0 and see what happens

And get index in the array for the minimum:

In [ ]:
lasso_min_mse = np.argmin(pd.DataFrame(lassoCV.mse_path_).mean(1))
lasso_min_mse

Which allow us to retrieve the best lambda:

In [ ]:
lassoCV.alphas_[lasso_min_mse]

As expected, it corresponds to the optimal value returned by `LassoCV.alpha`

### <a id='toc1_5_3_'></a>[Test error estimation](#toc0_)

Finally we can get the MSE and $R^2$ for the training and the test set. First, the training set:

In [ ]:
lassoCV.score(X_train,y_train) 

In [ ]:
# R2 can also be obtained as
r2_score(y_train,lassoCV.predict(X_train))

In [ ]:
mean_squared_error(y_train,lassoCV.predict(X_train))

Finally, the test set:

In [ ]:
lassoCV.score(X_test,y_test)

Or as

In [ ]:
r2_score(y_test,lassoCV.predict(X_test))

In [ ]:
mean_squared_error(y_test,lassoCV.predict(X_test))

We accumulate this values to our lists

In [52]:
model.append('LASSO')
r2_train.append(lassoCV.score(X_train,y_train))
r2_test.append(lassoCV.score(X_test,y_test))
mse_train.append(mean_squared_error(y_train,lassoCV.predict(X_train)))
mse_test.append(mean_squared_error(y_test,lassoCV.predict(X_test)))

Let us produce a final table:

In [ ]:
results = pd.DataFrame({'R2_train':r2_train,
                        'R2_test':r2_test,
                        'mse_train':mse_train,
                        'mse_test':mse_test},
                        index=model)
results

**Q**: Interpret the above results in terms of overfitting. 

## <a id='toc1_6_'></a>[YOUR TURN](#toc0_)

Estimate the regression using ElasticNetCV. In this example, we will cross-validate both the regularization penalty ($\lambda$) for three different values of the mix of Lasso-Ridge regularization.

1. Create three different Elastic Net instances with three values for (\alpha) (between 0 and 1).
2. Pass the set of ($\lambda$) values already created to these instances.
3. Complete the table with the results from these three models.
4. Reach a conclusion on the best predictive model.

## <a id='toc1_7_'></a>[YOUR TURN 2](#toc0_)

This is a more complex (and optional) task.

**Goal**: demonstrate the bias of LASSO regression in a high-dimensional linear regression. To do so implement the following steps:

**1. Bootstrapping the training sample:**
*   Perform the following steps a large number of times (e.g., B = 1000):
    +  Resample: Draw a random sample with replacement from the original dataset (bootstrapping).
    +  Fit LASSO: 
        *   Fit the LASSO regression model to the bootstrapped sample. 
        *   Determine the optimal tuning parameter (lambda) using cross-validation within the bootstrapped sample.
    +  Obtain Coefficients: Extract the estimated coefficients (β_hat_b) from the fitted LASSO model.

**2. Analyze the results:**
*   Empirical distribution:
    *   For each feature (j), you now have B estimates of its coefficient (β_hat_b,j). 
    *   Calculate the mean and standard deviation of these B estimates.
*   Compare to True Coefficients:
    *   Bias: Compare the mean of the bootstrapped coefficient estimates (mean(β_hat_b,j)) to the true coefficient (β_true,j). 
        *   If LASSO is biased, you'll observe systematic differences between the mean estimates and the true values.
    *   Variance: Examine the standard deviation of the bootstrapped coefficient estimates. This gives you an idea of the variability of the LASSO estimates.

**3. Visualization**:
*   Plot the results:
    *   Create scatter plots of the true coefficients (β_true) versus the mean of the bootstrapped coefficients (mean(β_hat_b)) for each feature. 
    *   Ideally, you'd see a strong linear relationship with a slope of 1 if LASSO were unbiased. Deviations from this line indicate bias.

**Key Considerations:**

*   **Sparsity:** The degree of sparsity in the true model will significantly impact LASSO's performance and bias (it can reduce bias for strong predictors and incerase bias for weak predictors).
*   **Signal-to-noise Ratio:** Higher noise levels can increase the bias of LASSO.
*   **Choice of tuning parameter:** The selection of the tuning parameter (lambda) in LASSO is crucial. Cross-validation within each bootstrap sample is recommended.
*   **Number of bootstrap replicates (B):** A larger number of bootstrap replicates will provide more stable estimates of bias and variability.


**Note:** While this approach demonstrates the potential for bias, it's important to remember that LASSO's primary advantage lies in its ability to perform variable selection and improve prediction accuracy in high-dimensional settings, even if it introduces some bias in the coefficient estimates.
